### Notes:

This version of the notebook is used for running the most recent version YOLO v5 which has feature maps to show which featuers are the most important. It can be used for re-training the model as needed.

In [1]:
import os
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets")
os.getcwd()

'D:\\Datasets\\SIIM_RSNA_Covid_Datasets'

In [2]:
# Set up OS and run to get YOLO v5 in the tmp folder

!mkdir tmp
%cd tmp
# %cd ../

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp


In [5]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Install W&B 
# !pip install -q --upgrade wandb
# Login 

import wandb
wandb.login()

# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

Setup complete. Using torch 1.8.1 (NVIDIA GeForce GTX 1080 Ti)


wandb: Currently logged in as: ogk01 (use `wandb login --relogin` to force relogin)


# 🦆 Hyperparameters

In [7]:
TRAIN_PATH = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\train_jpeg\\'
train_image_level_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\train_image_level.csv'
meta_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\meta_256.csv'
yolo_yaml_filepath = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5\\data\\data.yaml'
IMG_SIZE = 256
BATCH_SIZE = 256
EPOCHS = 20

# 🔨 Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.

I am using [xhlulu's](https://www.kaggle.com/xhlulu) resized dataset. The uploaded 256x256 Kaggle dataset is [here](https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg). Find other image resolutions [here](https://www.kaggle.com/c/siim-covid19-detection/discussion/239918).

* Create train-validation split. <br>
* Create required `/dataset` folder structure and more the images to that folder. <br>
* Create `data.yaml` file needed to train the model. <br>
* Create bounding box coordinates in the required YOLO format. 

In [8]:
# Everything is done from /kaggle directory.
#%cd ../

# Load image level csv file
df = pd.read_csv(train_image_level_filepath)

# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)

# Add absolute path
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)

# Get image level labels
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

df.head(5)

# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
meta_df = pd.read_csv(meta_filepath)
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']
train_meta_df.head(2)

# Merge both the dataframes
df = df.merge(train_meta_df, on='id',how="left")
df.head(2)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,D:\Datasets\SIIM_RSNA_Covid_Datasets\train_jpe...,opacity,3488,4256
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,D:\Datasets\SIIM_RSNA_Covid_Datasets\train_jpe...,none,2320,2832


## 🍘 Train-validation split

In [9]:
# Create train and validation split.
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df.image_level.values)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)

print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

Size of dataset: 6334, training images: 5067. validation images: 1267


D:\Anaconda\envs\Pytorch\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
D:\Anaconda\envs\Pytorch\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is: 

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `covid`.

In [10]:
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\")
os.makedirs('tmp/covid/images/train', exist_ok=True)
os.makedirs('tmp/covid/images/valid', exist_ok=True)

os.makedirs('tmp/covid/labels/train', exist_ok=True)
os.makedirs('tmp/covid/labels/valid', exist_ok=True)

#!ls tmp/covid/images

# Move the images to relevant split folder.
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\covid\\images\\train\\{row.id}.jpg')
    else:
        copyfile(row.path, f'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\covid\\images\\valid\\{row.id}.jpg')

100%|█████████████████████████████████████████████████████████████████████████████| 6334/6334 [00:51<00:00, 122.71it/s]


#### Copy the images into the tmp folder created above

## 🍜 Create `.YAML` file

The `data.yaml`, is the dataset configuration file that defines 

1. an "optional" download command/URL for auto-downloading, 
2. a path to a directory of training images (or path to a *.txt file with a list of training images), 
3. a path to a directory of validation images (or path to a *.txt file with a list of validation images), 
4. the number of classes, 
5. a list of class names.

> 📍 Important: In this competition, each image can either belong to `opacity` or `none` image-level labels. That's why I have  used the number of classes, `nc` to be 2. YOLOv5 automatically handles the images without any bounding box coordinates. 

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required. 

In [11]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '../covid/images/train',
    val = '../covid/images/valid',
    nc = 2,
    names = ['none', 'opacity']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open(yolo_yaml_filepath, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
#%cat tmp/yolov5/data/data.yaml

## 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with **bounding box(es)** a `.txt` file with the same name as the image will be created in the format shown below:

* One row per object. <br>
* Each row is class `x_center y_center width height format`. <br>
* Box coordinates must be in normalized xywh format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by image width, and `y_center` and `height` by image height. <br>
* Class numbers are zero-indexed (start from 0). <br>

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets. 

In [12]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

# Prepare the txt files for bounding box
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.id
    # Get split
    split = row.split
    # Get image-level label
    label = row.image_level
    
    if row.split=='train':
        file_name = f'tmp/covid/labels/train/{row.id}.txt'
    else:
        file_name = f'tmp/covid/labels/valid/{row.id}.txt'
        
    
    if label=='opacity':
        # Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row, bboxes)
        # Format for YOLOv5
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

100%|█████████████████████████████████████████████████████████████████████████████| 6334/6334 [00:15<00:00, 396.15it/s]


# 🚅 Train with W&B



#%cd tmp/yolov5/
```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5s.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [13]:
os.chdir("D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5")
os.getcwd()

'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\tmp\\yolov5'

In [14]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project kaggle-siim-covid

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=256, img_size=[256], rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=kaggle-siim-covid, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 


Plotting labels... 

autoanchor: Analyzing anchors... anchors/target = 5.77, Best Possible Recall (BPR) = 1.0000
                 all       1267       1582     0.0141     0.0714    0.00744    0.00121
Saving model artifact on epoch  1
                 all       1267       1582      0.052      0.135     0.0236    0.00432
Saving model artifact on epoch  2
                 all       1

YOLOv5  v5.0-295-g9dd33fd torch 1.8.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264.0MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
tensorboard: Start with 'tensorboard --logdir kaggle-siim-covid', view at http://localhost:6006/
wandb: Currently logged in as: ogk01 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.30
wandb: Syncing run exp
wandb:  View project at https://wandb.ai/ogk01/kaggle-siim-covid
wandb:  View run at https://wandb.ai/ogk01/kaggle-siim-covid/runs/3co6ia1q
wandb: Run data is saved local

      1/19     9.28G   0.09815   0.01824  0.006705    0.1231       561       256:  15%|#5        | 3/20 [00:01<00:08,  2.04it/s]
      1/19     9.28G   0.09803   0.01817  0.006486    0.1227       548       256:  15%|#5        | 3/20 [00:01<00:08,  2.04it/s]
      1/19     9.28G   0.09803   0.01817  0.006486    0.1227       548       256:  20%|##        | 4/20 [00:01<00:07,  2.02it/s]
      1/19     9.28G   0.09765   0.01803  0.006251    0.1219       527       256:  20%|##        | 4/20 [00:02<00:07,  2.02it/s]
      1/19     9.28G   0.09765   0.01803  0.006251    0.1219       527       256:  25%|##5       | 5/20 [00:02<00:07,  2.03it/s]
      1/19     9.28G   0.09718   0.01817  0.006069    0.1214       555       256:  25%|##5       | 5/20 [00:02<00:07,  2.03it/s]
      1/19     9.28G   0.09718   0.01817  0.006069    0.1214       555       256:  30%|###       | 6/20 [00:02<00:06,  2.03it/s]
      1/19     9.28G   0.09663   0.01847  0.005896     0.121       590       256:  30%|###       


  0%|          | 0/20 [00:00<?, ?it/s]
      7/19     9.28G   0.06447   0.01952 0.0006291   0.08462       593       256:   0%|          | 0/20 [00:00<?, ?it/s]
      7/19     9.28G   0.06447   0.01952 0.0006291   0.08462       593       256:   5%|5         | 1/20 [00:00<00:09,  2.01it/s]
      7/19     9.28G   0.06652   0.01863 0.0006249   0.08577       565       256:   5%|5         | 1/20 [00:00<00:09,  2.01it/s]
      7/19     9.28G   0.06652   0.01863 0.0006249   0.08577       565       256:  10%|#         | 2/20 [00:00<00:08,  2.01it/s]
      7/19     9.28G   0.06666   0.01849 0.0006306   0.08578       549       256:  10%|#         | 2/20 [00:01<00:08,  2.01it/s]
      7/19     9.28G   0.06666   0.01849 0.0006306   0.08578       549       256:  15%|#5        | 3/20 [00:01<00:08,  2.00it/s]
      7/19     9.28G   0.06688   0.01839 0.0006324    0.0859       573       256:  15%|#5        | 3/20 [00:01<00:08,  2.00it/s]
      7/19     9.28G   0.06688   0.01839 0.0006324    0.0859     

## Model Saved Automatically as Artifact

Since it's a kernel based competition, you can easily download the best model from the W&B Artifacts UI and upload as a Kaggle dataset that you can load in your inference kernel (internel disabled).

### [Path to saved model $\rightarrow$](https://wandb.ai/ayush-thakur/kaggle-siim-covid/artifacts/model/run_jbt74n7q_model/4c3ca5752dba99bd227e)

![img](https://i.imgur.com/KhRLQvR.png)

> 📍 Download the model with the `best` alias tagged to it. 

# Inference

You will probably use a `Submission.ipynb` kernel to run all the predictions. After training a YOLOv5 based object detector -> head to the artifacts page and download the best model -> upload the model as a Kaggle dataset -> Use it with the submission folder. 

> 📍 Note that you might have to clone the YOLOv5 repository in a Kaggle dataset as well. 

In this section, I will show you how you can do the inference and modify the predicted bounding box coordinates.

In [18]:
# absolute path
TEST_PATH = 'D:\\Datasets\\SIIM_RSNA_Covid_Datasets\\test_jpeg'

Since I am training the model in this kernel itself, I will not be using the method that I have described above. The best model is saved in the directory `project_name/exp*/weights/best.pt`. In `exp*`, * can be 1, 2, etc. 

In [16]:
MODEL_PATH = 'kaggle-siim-covid/exp/weights/best.pt'

```
--weights {MODEL_PATH} \ # path to the best model.
--source {TEST_PATH} \ # absolute path to the test images.
--img {IMG_SIZE} \ # Size of image
--conf 0.281 \ # Confidence threshold (default is 0.25)
--iou-thres 0.5 \ # IOU threshold (default is 0.45)
--max-det 3 \ # Number of detections per image (default is 1000) 
--save-txt \ # Save predicted bounding box coordinates as txt files
--save-conf # Save the confidence of prediction for each bounding box
```

In [19]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf \
                  --visualize

YOLOv5  v5.0-295-g9dd33fd torch 1.8.1 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264.0MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients, 16.3 GFLOPs
D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5\utils\plots.py:465: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(math.ceil(n / 8), 8, tight_layout=True)  # 8 rows x n/8 cols
Traceback (most recent call last):
  File "detect.py", line 228, in <module>
    main(opt)
  File "detect.py", line 223, in main
    run(**vars(opt))
  File "D:\Anaconda\envs\Pytorch\lib\site-packages\torch\autograd\grad_mode.py", line 27, in decorate_context


detect: weights=['kaggle-siim-covid/exp/weights/best.pt'], source=D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg, imgsz=256, conf_thres=0.281, iou_thres=0.5, max_det=3, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=True, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
image 1/1263 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg\0026720152f5.jpg: Saving runs\detect\exp2\0026720152f5\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\0026720152f5\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\0026720152f5\stage2_C3_features.png... (32/64)
Saving runs\detect\exp2\0026720152f5\stage3_Conv_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage4_C3_features.png... (32/128)
Saving runs\detect\exp2\0026720152f5\stage5_Conv_features.png... (32/256)
Saving runs\detect\exp2\0026720152f5\

    return func(*args, **kwargs)
  File "detect.py", line 104, in run
    pred = model(img,
  File "D:\Anaconda\envs\Pytorch\lib\site-packages\torch\nn\modules\module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5\models\yolo.py", line 123, in forward
    return self.forward_once(x, profile, visualize)  # single-scale inference, train
  File "D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5\models\yolo.py", line 158, in forward_once
    feature_visualization(x, m.type, m.i, save_dir=visualize)
  File "D:\Datasets\SIIM_RSNA_Covid_Datasets\tmp\yolov5\utils\plots.py", line 473, in feature_visualization
    plt.savefig(save_dir / f, dpi=300, bbox_inches='tight')
  File "D:\Anaconda\envs\Pytorch\lib\site-packages\matplotlib\pyplot.py", line 859, in savefig
    res = fig.savefig(*args, **kwargs)
  File "D:\Anaconda\envs\Pytorch\lib\site-packages\matplotlib\figure.py", line 2311, in savefig
    self.canvas.print_


Saving runs\detect\exp2\022146012034\stage13_C3_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage14_Conv_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\022146012034\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\022146012034\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\022146012034\stage23_C3_features.png... (32/512)
256x256 Done. (19.383s)
image 6/1263 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg\02eceb0fc405.jpg: Saving runs\detect\exp2\02eceb0fc405\stage0_Focus_features.png... (32/

Saving runs\detect\exp2\198ffac65dac\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\198ffac65dac\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\198ffac65dac\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\198ffac65dac\stage22_Concat_features.png... (32/512)
Saving runs\detect\exp2\198ffac65dac\stage23_C3_features.png... (32/512)
256x256 Done. (18.264s)
image 128/1263 D:\Datasets\SIIM_RSNA_Covid_Datasets\test_jpeg\19b0d2d65a38.jpg: Saving runs\detect\exp2\19b0d2d65a38\stage0_Focus_features.png... (32/32)
Saving runs\detect\exp2\19b0d2d65a38\stage1_Conv_features.png... (32/64)
Saving runs\detect\exp2\19b0d2d65a38\stage2_C3_features.png... (32/64)
Saving runs\detect\exp2\19b0d2d65a38\stage3_Conv_features.png... (32/128)
Saving runs\detect\exp2\19b0d2d65a38\stage4_C3_features.png... (32/128)
Saving runs\detect\exp2\19b0d2d65a38\stage5_Conv_features.png... (32/256)
Saving runs\detect\exp2\19b0d2d65a38\stage6_C3_features.png... (32/256)
Saving runs

Saving runs\detect\exp2\340c9cf3b19a\stage10_Conv_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage11_Upsample_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage12_Concat_features.png... (32/512)
Saving runs\detect\exp2\340c9cf3b19a\stage13_C3_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage14_Conv_features.png... (32/128)
Saving runs\detect\exp2\340c9cf3b19a\stage15_Upsample_features.png... (32/128)
Saving runs\detect\exp2\340c9cf3b19a\stage16_Concat_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage17_C3_features.png... (32/128)
Saving runs\detect\exp2\340c9cf3b19a\stage18_Conv_features.png... (32/128)
Saving runs\detect\exp2\340c9cf3b19a\stage19_Concat_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage20_C3_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage21_Conv_features.png... (32/256)
Saving runs\detect\exp2\340c9cf3b19a\stage22_Concat_features.png... (32/512)
Saving runs\det

In [15]:
# # Original Code
# !python detect.py --weights {MODEL_PATH} \
#                   --source {TEST_PATH} \
#                   --img {IMG_SIZE} \
#                   --conf 0.281 \
#                   --iou-thres 0.5 \
#                   --max-det 3 \
#                   --save-txt \
#                   --save-conf

usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--imgsz IMGSZ] [--conf-thres CONF_THRES]
                 [--iou-thres IOU_THRES] [--max-det MAX_DET] [--device DEVICE]
                 [--view-img] [--save-txt] [--save-conf] [--save-crop]
                 [--nosave] [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--visualize] [--update] [--project PROJECT]
                 [--name NAME] [--exist-ok] [--line-thickness LINE_THICKNESS]
                 [--hide-labels] [--hide-conf] [--half]
detect.py: error: argument --imgsz/--img/--img-size: invalid int value: '{IMG_SIZE}'


### How to find the confidence score?

1. First first the [W&B run page](https://wandb.ai/ayush-thakur/kaggle-siim-covid/runs/jbt74n7q) generated by training the YOLOv5 model. 

2. Go to the media panel -> click on the F1_curve.png file to get a rough estimate of the threshold -> go to the Bounding Box Debugger panel and interactively adjust the confidence threshold. 

![img](https://i.imgur.com/cCUnTBw.gif)

> 📍 The bounding box coordinates are saved as text file per image name. It is saved in this directory `runs/detect/exp3/labels`. 

In [136]:
PRED_PATH = 'runs/detect/exp3/labels'
!ls {PRED_PATH}

00fc8fc35dc1.txt  409322480a9b.txt  858667501a67.txt  bca76008a759.txt
03a778f5a68b.txt  413344e1032c.txt  85b6452b47cc.txt  be4b4301c5e4.txt
042979d394a1.txt  445298de90b9.txt  86734b7cc20f.txt  be53bfb563b3.txt
04396dd5af2f.txt  458417ad6155.txt  87a0829f53c1.txt  beb80bb88a6e.txt
04720d7e4e42.txt  4810ae5dbc08.txt  87c51db67bf7.txt  bf30cc575dc9.txt
049fce8128f9.txt  4905b651a091.txt  87d8baf120a6.txt  bf5c7ef3b72a.txt
04de800ea41a.txt  49c1b06028a1.txt  8829d22c3e2b.txt  bf8e59f80d5c.txt
04f4a36c08a2.txt  49e9379c42dc.txt  88782677cbec.txt  c02ef6a70c7e.txt
05435da60872.txt  49fb3141efe3.txt  89015a2258aa.txt  c0a746ef9f9f.txt
0562436fde50.txt  4b0058c1f54e.txt  8927235865c9.txt  c0f5131b1c99.txt
05a70a1c16c1.txt  4c2d931d0f33.txt  898d24f3cda6.txt  c263b1e9aa64.txt
05feea9719f5.txt  4d56b2fc668c.txt  89ee5be93326.txt  c28570a073d9.txt
06df1e2fde68.txt  4d6c8e93e7c9.txt  8b354d4a216f.txt  c39146cbda47.txt
07db3756ec28.txt  4d7b06512cc1.txt  8b6ebd55c8c8.txt  c3c99fea8c77.txt
08992b